## Assignment 3
### Name: Aagaaz Ali Sayed
### Roll NO: 43369
### Batch: S11

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

2.9.2


### 1. Loading and Preprocessing the image data

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 3s 0us/step


In [ ]:
print(f"train data shape: {x_train.shape}")
print(f"label train shape: {y_train.shape}")
print(f"test data shape: {x_test.shape}")
print(f"label test shape: {y_test.shape}")

train data shape: (50000, 32, 32, 3)
label train shape: (50000, 1)
test data shape: (10000, 32, 32, 3)
label test shape: (10000, 1)


In [ ]:
y_train[0]

array([6], dtype=uint8)

In [ ]:
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

In [ ]:
print(f"label train shape: {y_train.shape}")
print(f"label test shape: {y_test.shape}")

label train shape: (50000, 10)
label test shape: (10000, 10)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def get_generator_aug():
    data_generator_aug = ImageDataGenerator(rescale=(1/255.0),
                                       rotation_range=35,
                                       width_shift_range=0.3,
                                       height_shift_range=0.3,
                                       fill_mode='nearest',
                                       brightness_range=(0.2,0.8),
                                       shear_range=45.0,
                                       horizontal_flip=True,
                                       vertical_flip=True,
                                       zoom_range=[0.5, 1.5])
    return data_generator_aug

In [ ]:
data_generator_aug = get_generator_aug()
data_generator_aug.fit(x_train)
train_generator_aug = data_generator_aug.flow(x_train, y_train, batch_size=10, shuffle=True)

data_generator_aug_test = get_generator_aug()
data_generator_aug_test.fit(x_test)
train_generator_aug = data_generator_aug.flow(x_train, y_train, batch_size=10, shuffle=True)

In [ ]:
data_generator = ImageDataGenerator(rescale=(1/255.0))
data_generator.fit(x_train)
img_generator = data_generator.flow(x_train, y_train, batch_size=10, shuffle=False)

### 2. Defining model architecture

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model

In [ ]:
def get_model(input_shape):
  input_layer = Input(input_shape)
  layer1 = Conv2D(32, 8, activation='relu', padding='SAME')(input_layer)
  layer2 = MaxPooling2D((2,2))(layer1)
  layer3 = Conv2D(32, 4, activation='relu', padding='SAME')(layer2)
  layer4 = MaxPooling2D((2,2))(layer3)
  layer5 = Flatten()(layer4)
  layer6 = Dense(16, activation="relu")(layer5)
  output_layer = Dense(10, activation='softmax')(layer6)

  model = Model(inputs=input_layer, outputs=output_layer)

  model.compile(optimizer=tf.keras.optimizers.Adam(3e-4),
                loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy()])
  return model

In [ ]:
model = get_model((32,32,3))
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        6176      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        16416     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 2048)              0     

### 3. Training the model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

train_steps_per_epoch = img_generator.n // img_generator.batch_size

In [ ]:
history = model.fit(img_generator, steps_per_epoch=train_steps_per_epoch, validation_data=(x_test,y_test),epochs=50, callbacks=[EarlyStopping(monitor='val_categorical_accuracy', patience=5)])

Epoch 1/50
5000/5000 [==============================] - 192s 38ms/step - loss: 1.6921 - categorical_accuracy: 0.3805 - val_loss: 162.4378 - val_categorical_accuracy: 0.4434
Epoch 2/50
5000/5000 [==============================] - 195s 39ms/step - loss: 1.3882 - categorical_accuracy: 0.4936 - val_loss: 167.2987 - val_categorical_accuracy: 0.4693
Epoch 3/50
5000/5000 [==============================] - 186s 37ms/step - loss: 1.2606 - categorical_accuracy: 0.5439 - val_loss: 166.1182 - val_categorical_accuracy: 0.5102
Epoch 4/50
5000/5000 [==============================] - 186s 37ms/step - loss: 1.1822 - categorical_accuracy: 0.5742 - val_loss: 185.4658 - val_categorical_accuracy: 0.4976
Epoch 5/50
5000/5000 [==============================] - 186s 37ms/step - loss: 1.1215 - categorical_accuracy: 0.5964 - val_loss: 188.6397 - val_categorical_accuracy: 0.5097
Epoch 6/50
5000/5000 [==============================] - 187s 37ms/step - loss: 1.0706 - categorical_accuracy: 0.6170 - val_loss: 214.50

### 4. Evaluating its performance

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
y_pred = model.predict(x_test)

313/313 [==============================] - 9s 28ms/step


In [ ]:
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
y_test = np.argmax(y_test, axis=1)

In [ ]:
print(accuracy_score(y_test, y_pred))

0.5293
